In [3]:
import FlimObj
from tkinter import filedialog
import re, glob, os
import matplotlib.pyplot as plt
import datetime
import numpy as np
import tkinter as Tk
import pandas as pd
import sys, getopt

# I KNOW I KNOW. but the solver starts to make a lot of noise when the estimates start to converge.
import warnings
warnings.filterwarnings("ignore")

def roi_analysis(flimobj,roi_im,tauo=None):
	if tauo:
		_, emp_taus = flimobj.fit_tau(roi_im,tauo=tauo, avg=1)
	else:
		_,emp_taus = flimobj.fit_tau(roi_im,avg=1)

	return emp_taus

def flim_analysis(file_path, rois = None, t_base=None, tauo = None, channel=0, average=1):
	# compute dfof for each ROI
	## copied from stitch_t.py
	flimobj = FlimObj.FlimObj()
	flimobj.read_FLIM(file_path)
	flimobj.times_to_datetime() # adds an attribute to the flimobj that stores the times as datetimes (flimobj.datetimes)
	image_array = flimobj.get_images_as_intensity_array(flimbins=True)
	frames = image_array.shape[0]
	image_array = np.add.reduceat(image_array,np.arange(0,frames,average),axis=0) # sum merged images
	print(image_array.shape)
	# fit tauo if none
	if not tauo:
		flimobj.fit_tau(np.sum(image_array,axis=1,keepdims=True),avg=1,channel=channel)
		tauo = flimobj.tauo

	## Fix the t axis if there's a pulse
	x_in_seconds = flimobj.get_t_axis() # it's in seconds unless you specify otherwise
	pulse_t = flimobj.get_pulse_delay()

	pulse_frame = int(np.sum(x_in_seconds<pulse_t) + flimobj.get_pulse_frames_baseline())
	pulse_frame = int(pulse_frame / average)
	if t_base:
		t_offset = (flimobj.datetimes[0]-t_base).total_seconds()
		x_in_seconds = x_in_seconds+t_offset
	else:
		x_in_seconds = x_in_seconds-pulse_t
		try:
			t_base = flimobj.datetimes[pulse_frame]
		except:
			print("Pulse parameters outside of acquired data! Double check to make sure analysis is right.")
			t_base = flimobj.datetimes[0]	
	
	

	xmax = image_array.shape[3]
	ymax = image_array.shape[4]
	


	# pick the time point when averaging
	if average > 1: # pool together time bins if averaging time bins
		x_in_seconds = x_in_seconds[::average]# take first time point

	## ROI SPECIFIC ANALYSIS HERE
	# rois is a list of tuples of the x and y bounds of each ROI, and the z value. Structure: [(roi,z),(roi,z)]. Structure of roi: ((x0,y0),(x1,y1))
	emp_taus_list = []

	for roi_pair in rois:
		roi, z_val = roi_pair[0], roi_pair[1]
		xbounds = np.sort([np.min((roi[0][0],xmax)),np.min((roi[1][0],xmax))])
		ybounds = np.sort([np.min((roi[0][1],ymax)),np.min((roi[1][1],ymax))]) # make sure the bounds don't get messed up
		print("Evaluating ROI!")
		roi_im = np.expand_dims(image_array[:,z_val,:,ybounds[0]:ybounds[1],xbounds[0]:xbounds[1],...],axis=1) # UGH the axis reversal
		print(roi_im.shape)
		emp_taus = roi_analysis(flimobj,roi_im,tauo=tauo) # apply analysis to the ROI
		print(emp_taus)
		emp_taus_list.append(emp_taus)


	return x_in_seconds, emp_taus_list, tauo, rois, flimobj

def pulsed_t_series(t_axis, dfof, flimobj,average=1):
	# handle the pulses by turning them into nans, return the times of pulses
	pulse_params = flimobj.FLIMageFileReader.State.Uncaging
	# params (in milliseconds)
	(nPulses, pulsewidth, isi, delay) = (pulse_params.nPulses, pulse_params.pulseWidth, pulse_params.pulseISI, pulse_params.pulseDelay)
	pidx_list = []
	p_offset = pulse_params.baselineBeforeTrain_forFrame
	for p in range(nPulses):
		pstart = delay + p*(isi)+p_offset
		pend = p*(isi+pulsewidth)+delay + pulsewidth+p_offset # in milliseconds
		# now measure from start to end of each pulse
		pidxs = np.floor(np.arange(flimobj.last_frame_before_time(pstart, units="milliseconds")-1,\
			flimobj.last_frame_before_time(pend, units="milliseconds"),average)/average).astype(int)
		for j in range(len(dfof)):
			dfof[j][pidxs] = np.nan
		pidx_list.append(pidxs)
	return dfof, pidx_list

def single_round_process(filename,plot=False, tauo=None, t_base=None, channel=0, average=1, rois = None):
	# everything I want to do to a single trace
	t_axis, emp_taus, tauo, rois, flimobj = flim_analysis(filename, tauo=tauo, t_base=t_base, channel=channel, average=average, rois = rois) # the flimobj contains a lot of additional info
	
	# if there are pulses
	if flimobj.FLIMageFileReader.State.Uncaging.uncage_whileImage:
		emp_taus, pidxs = pulsed_t_series(t_axis, emp_taus, flimobj,average=average) # deal with the pulses, find out which frames correspond to them
		if not t_base:
			t_axis = t_axis - t_axis[pidxs[0].ravel()[0]]
	if plot:
		for roi in emp_taus:
			plt.plot(t_axis,np.array(roi))
	return t_axis, emp_taus,tauo, t_axis[pidxs[0].ravel()[0]]


In [42]:
def main(argv):
    # extract number of frames to average, other params
    unixOpts = "a:c:s"
    gnuOpts = ["average=","channels=","save="]
    try:
        avg=1
        channels=1
        save=False
        arguments, values = getopt.getopt(argv, unixOpts, gnuOpts)
        for opt, arg in arguments:
            print(opt,arg)
            if opt in ('-a',"--average"):
                avg = int(arg)
            elif opt in ('-s', '--save'):
                save = True
            elif opt in ('-c', '--channels'):
                channels = int(arg)
    except getopt.GetoptError as err:
        print(str(err))
        usage()
        sys.exit(2)

    # GUI stuff
    root = Tk.Tk()
    root.withdraw()
    #direc = filedialog.askdirectory()
    root.update()
    file_path = filedialog.askopenfilename()
    root.update()
    try:
        basename = re.match("(.*\D)[0-9]+\.flim",file_path).group(1)  # up to the part that ends in <numbers>.flim
    except: # file not selected
        sys.exit(0)

	### find files

    filelist = sorted(glob.glob("%s*.flim"%(basename)),key=lambda x: int(re.findall("(\d+)",x)[-1])) # sort numerically

	### preparatory arrays for the time axis, the tau axis, etc.

    t_list = [[] for c in range(channels)]
    tau_list = [[] for c in range(channels)]
    tauo_list = [None for c in range(channels)]
    t_base = [None for c in range(channels)]

    ### Define rois
    roi_obj = FlimObj.FlimObj()
    roi_obj.read_FLIM(filelist[0]) # select roi from first frames
    image_array = roi_obj.get_images_as_intensity_array(flimbins=False)
    print(image_array.shape)
    # contains an attribute roi_coords with the coordinates for each ROI
    image_scroller = roi_obj.viz_frame(image_array,axes_dict={'t':0,'z':1,'c':2,'x':3,'y':4})
    roi_coords = image_scroller.roi_coords # format is list of length 1:z_max containing roi for each z plane in format ((x0,y0),(x1,y1))
    rois = [(roi,z) for z in range(len(roi_coords)) for roi in roi_coords[z]]

#	Try it for yourself, it's the stupid index reversal. Not sure why it works that way.
#	xmax = image_array.shape[3]
#	ymax = image_array.shape[4]
#	for pair in rois:
#		roi, z = pair[0], pair[1]
#		xbounds = np.sort([np.min((roi[0][0],xmax)),np.min((roi[1][0],xmax))])
#		ybounds = np.sort([np.min((roi[0][1],ymax)),np.min((roi[1][1],ymax))]) # make sure the bounds don't get messed up
#		plt.figure()
#		plt.imshow(image_array[0,z,0,ybounds[0]:ybounds[1],xbounds[0]:xbounds[1]])
#	plt.show()

### iterate through each file, perform analysis

    for file in filelist:
        for c in range(channels): # iterate through color channels
            if not tauo_list[c]: # check if empty
                t_axis, emp_taus, tauo, tb = single_round_process(file, channel=c, average=avg, rois=rois,plot=False) # process each trace
                tauo_list[c] = tauo
                t_base[c] = tb
            else:
                t_axis,emp_taus, _,_ = single_round_process(file, tauo=tauo_list[c], t_base=t_base[c], average=avg, rois=rois, plot=False)
            t_list[c].append(t_axis)
            tau_list[c].append(emp_taus)

    ### merging all the arrays from each individual file
    t_axis = [np.concatenate(t_list[c], axis = 0).flatten() for c in range(channels)]
    print(t_axis)
    taus = [np.concatenate(tau_list[c], axis = 1).T for c in range(channels)]
    print(taus)
    return t_axis, taus

In [43]:
t, taus = main('-c 2')

(400, 1, 2, 128, 128)
Clicked!
Clicked!
(400, 1, 2, 128, 128, 64)
[8.87751035e-01 1.97694394e+01 1.00004595e+00 6.55929507e-07
 1.39999967e+01]
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.3327492  2.28102194 2.40403691 2.42643793 2.36379237 2.35896841
 2.38849587 2.40200066 2.34774101 2.36668139 2.33688591 2.41715773
 2.36698374 2.33050291 2.36796183 2.38245681 2.37035284 2.31417071
 2.36035346 2.31595235 2.39459083 2.40557444 2.38961313 2.42968691
 2.40839227 2.4197327  2.39966926 2.3515479  2.44485597 2.29737556
 2.41106985 2.356484   2.3559893  2.37853448 2.42031254 2.46621254
 2.43849273 2.40378488 2.42387638 2.3979999  2.33428525 2.38451982
 2.37857768 2.38661186 2.35869045 2.38219962 2.3854651  2.38720578
 2.34690125 2.37512581 2.34845182 2.41762726 2.42694075 2.39013379
 2.36829241 2.40443129 2.38040946 2.30691453 2.3767248  2.32827005
 2.3581932  2.43361736 2.35319719 2.39033104 2.38668821 2.38800225
 2.35108258 2.35295603 2.38731584 2.36145577 2.40707614 2.46320186
 2.2971304  

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.42876909 2.38544645 2.31235748 2.36326797 2.35747303 2.32910195
 2.37630934 2.39367001 2.37028099 2.36866558 2.32101569 2.38477858
 2.36948641 2.35722044 2.39142235 2.33184749 2.33400353 2.36811033
 2.37398952 2.39042011 2.3606275  2.4139632  2.37962118 2.39878382
 2.29013597 2.33824306 2.41964092 2.41047314 2.35264744 2.36656354
 2.44742627 2.40254873 2.38352951 2.33233008 2.39411831 2.38949163
 2.32031169 2.35785133 2.40716385 2.38518585 2.40459617 2.3553889
 2.35753883 2.37667595 2.43731616 2.38082896 2.295261   2.32872154
 2.39536728 2.36692448 2.40510747 2.39901979 2.43125367 2.42306476
 2.37668124 2.34238887 2.35478286 2.36369688 2.42707803 2.38398728
 2.39498239 2.39372432 2.3787458  2.40739704 2.4265218  2.34165915
 2.35554578 2.38035524 2.37471711 2.35691322 2.31355999 2.4187108
 2.40815143 2.37944904 2.34561343 2.39067374 2.35552392 2.37515389
 2.31088698 2.34967016 2.33474313 2.3540088  2.29724663 2.3062371

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.38746267 2.42208977 2.38676234 2.40756496 2.38289295 2.32990063
 2.40697307 2.41799917 2.37671095 2.34476607 2.37753783 2.37382541
 2.42628788 2.36953512 2.36960778 2.38342035 2.44918346 2.37714462
 2.36715576 2.35419019 2.36958256 2.38621411 2.37057204 2.39135305
 2.36214103 2.42776607 2.39627442 2.40832619 2.35833096 2.42350104
 2.37206333 2.37411387 2.43882946 2.41820924 2.33452762 2.38456516
 2.38838924 2.35538264 2.40883796 2.3724068  2.39269556 2.35753387
 2.4300702  2.33475323 2.39527332 2.37321728 2.38782937 2.3530374
 2.39428956 2.35165424 2.38660679 2.38125185 2.33643297 2.36115359
 2.43450272 2.32525664 2.39490226 2.35076588 2.43689722 2.3713803
 2.3877397  2.42075433 2.38883035 2.36128252 2.40543404 2.33088497
 2.36414777 2.38378646 2.4312541  2.38248131 2.42116935 2.30716879
 2.37062566 2.32009779 2.35892684 2.38757636 2.35679362 2.36891958
 2.40606927 2.34087121 2.42998388 2.38512897 2.26758005 2.3822529

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.3668858  2.36645776 2.38796186 2.38680148 2.39779643 2.40677503
 2.35095434 2.36448178 2.34026568 2.38994223 2.35518592 2.36211583
 2.37901182 2.36406919 2.38948878 2.36767449 2.37461202 2.40576863
 2.41469667 2.41771716 2.35754364 2.34251079 2.38807781 2.33589187
 2.4105607  2.38752877 2.41946778 2.37067021 2.37998658 2.33922463
 2.38786552 2.37689414 2.35046805 2.35713049 2.39209625 2.38109957
 2.36391429 2.4002927  2.37402718 2.37315601 2.37821184 2.41735197
 2.32125213 2.40579591 2.42209319 2.38768451 2.34464779 2.36485542
 2.36191596 2.38374997 2.35962964 2.35816302 2.3836929  2.40380286
 2.34440186 2.40993141 2.38722563 2.35514259 2.3572782  2.32800066
 2.3455979  2.37854271 2.38210274 2.3328939  2.38363618 2.39512378
 2.3791804  2.4417472  2.41626425 2.34956588 2.38409322 2.36515779
 2.38225755 2.39560184 2.39671749 2.38166836 2.36696135 2.45047887
 2.35503609 2.41158719 2.36751901 2.37333737 2.35310134 2.39083

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.37577322 2.38405413 2.36166707 2.38722418 2.36005719 2.34590079
 2.37952042 2.37496198 2.33924543 2.38153339 2.35302399 2.39010779
 2.36040565 2.37631365 2.36130908 2.38026073 2.4441915  2.3648573
 2.34951217 2.34896574 2.38407661 2.38478258 2.36199006 2.4048864
 2.37187817 2.4271694  2.35938481 2.39362144 2.4226469  2.36356315
 2.34178907 2.42381167 2.37721187 2.3829329  2.36782787 2.39525327
 2.40804273 2.36051163 2.33504618 2.3628647  2.36026247 2.38128978
 2.39902521 2.36265013 2.38089279 2.36326212 2.37489731 2.39458002
 2.34074845 2.3055206  2.37647667 2.37773727 2.3734847  2.3534766
 2.36856098 2.36394025 2.32644824 2.40980732 2.34099201 2.38841058
 2.39286345 2.3485361  2.41246861 2.41301012 2.35021873 2.37251802
 2.32309233 2.39017352 2.34784461 2.40675998 2.44344993 2.41161357
 2.41244205 2.35889666 2.37571518 2.424905   2.34674548 2.38304707
 2.38994426 2.40358388 2.36667785 2.36935576 2.33370766 2.43141878

...

[2.36289662 2.34946153 2.36880469 2.37277672 2.37227258 2.37237653
 2.35709066 2.38422504 2.35473099 2.35893521 2.37674728 2.35039955
 2.36767515 2.37391349 2.34765386 2.37103601 2.36291975 2.37421357
 2.38371329 2.380854   2.35279618 2.37151057 2.359662   2.37830262
 2.3668742  2.3642843  2.37650543 2.36770183 2.38675006 2.34149383
 2.38765362 2.36292198 2.36548177 2.3748908  2.34627301 2.34903539
 2.37170378 2.34293006 2.35515405 2.38194759 2.36148114 2.35588259
 2.37176565 2.3519817  2.37520408 2.36225868 2.36614643 2.359894
 2.3592281  2.36819937 2.37544117 2.36030124 2.34455548 2.37342831
 2.36616979 2.36602263 2.3503057  2.35544108 2.37393897 2.33944271
 2.36866668 2.37124467 2.36839571 2.38064446 2.3675759  2.36242822
 2.36625338 2.35915317 2.3822243  2.37892594 2.35719012 2.37102096
 2.36777266 2.36357708 2.34831917 2.34799893 2.35972015 2.34403669
 2.35885183 2.37056338 2.35328763 2.36692403 2.38827839 2.36171806
 2.39017268 2.35220958 2.37462304 2.37238658 2.36310876 2.372041

[2.40330722 2.36599246 2.34481288 2.36432636 2.37164196 2.35273154
 2.38429597 2.34419513 2.34378719 2.39071747 2.38709144 2.37237929
 2.36416632 2.36070799 2.37725925 2.38716607 2.39109046 2.38210608
 2.37059795 2.34349945 2.35872902 2.36453905 2.33989239 2.38138002
 2.38919901 2.37426569 2.41571444 2.33740297 2.36625137 2.3959607
 2.36806874 2.37354525 2.37945918 2.37781806 2.42398026 2.34818116
 2.39448101 2.37045624 2.35151443 2.37475856 2.38541556 2.36887484
 2.37649264 2.36337687 2.37271848 2.3502175  2.34859751 2.37308714
 2.35034305 2.36363703 2.35567437 2.35786025 2.37723181 2.40209792
 2.36544902 2.35438636 2.37541872 2.3863276  2.36455317 2.38802663
 2.36068681 2.39496046 2.37414598 2.39261406 2.34534861 2.36827798
 2.32605749 2.3748491  2.38260032 2.39039244 2.33389447 2.35704014
 2.37157843 2.34011281 2.41085183 2.34127136 2.34821561 2.3909181
 2.35440773 2.40132233 2.36258167 2.35826897 2.33948946 2.37135073
 2.39055949 2.37419784 2.38176803 2.35936999 2.40585912 2.364753

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.39681094 2.34974035 2.37494616 2.38415868 2.37020751 2.44590428
 2.3557699  2.43750326 2.39529574 2.42997387 2.42821728 2.38295498
 2.30731578 2.38813016 2.44411433 2.42269625 2.39638691 2.36886785
 2.35477294 2.36761085 2.42792182 2.41851995 2.36290389 2.37706102
 2.34588302 2.36545233 2.34464713 2.43856118 2.38576905 2.35334892
 2.41945331 2.39457174 2.35953639 2.33225097 2.46529932 2.40236512
 2.44131656 2.37623467 2.41680502 2.33294279 2.35806651 2.38966147
 2.43671941 2.40244931 2.34451173 2.39943592 2.41292611 2.36368978
 2.39125489 2.43297676 2.35857123 2.41460943 2.40441968 2.44317527
 2.36970003 2.36963645 2.39225952 2.38864019 2.42023176 2.3741169
 2.3578436  2.34187889 2.41964822 2.38383623 2.35771009 2.35645131
 2.35132425 2.33795672 2.37040534 2.37619888 2.35198126 2.36425817
 2.37169917 2.40961127 2.32805163 2.38794848 2.33385703 2.35246559
 2.39942524 2.38738365 2.31487927 2.3772679  2.3849451  2.431835

(400, 1, 2, 128, 128, 64)
Evaluating ROI!
(400, 1, 2, 39, 69, 64)
[2.34799238 2.37111965 2.37508647 2.39962676 2.36602897 2.37103543
 2.36718333 2.3884583  2.37790886 2.43280956 2.41726655 2.36318163
 2.40591464 2.37828028 2.43339064 2.38538138 2.33675245 2.40558588
 2.38025056 2.39746797 2.38230155 2.41725077 2.35873016 2.36467469
 2.29833435 2.33640041 2.38061313 2.37442877 2.33980405 2.41858257
 2.37484779 2.36083983 2.43235966 2.41394826 2.36716363 2.3865973
 2.38383313 2.44752501 2.41793141 2.39840335 2.40097607 2.44369284
 2.36501415 2.36789171 2.41342291 2.39376375 2.38091954 2.36445723
 2.40703239 2.35913446 2.36687366 2.38519043 2.3622775  2.3932224
 2.37022585 2.44513969 2.38516997 2.34116958 2.36921492 2.40883162
 2.41360951 2.37373089 2.33164993 2.36482915 2.38207419 2.40828443
 2.42189016 2.42337023 2.37754438 2.35296861 2.40317527 2.29860071
 2.35806382 2.35918434 2.36745012 2.36097939 2.38312659 2.36651747
 2.39755851 2.35674687 2.40551529 2.38149244 2.41353675 2.3972382

[array([-30.72 , -30.464, -30.208, ...,  70.912,  71.168,  71.424])]
[array([[2.3327492 , 2.35627162],
       [2.28102194, 2.35128186],
       [2.40403691, 2.38613991],
       ...,
       [2.42240747, 2.3769083 ],
       [2.36667412, 2.34188158],
       [2.3973744 , 2.3877426 ]])]


In [47]:
for c in range(1):
    print(taus[0].shape)

(8000, 2)


In [63]:
print(np.hstack((t[0][:,np.newaxis],taus[0])).shape)

(8000, 3)
